In [ ]:
# !pip install Selenium
# !apt-get update
# !apt install chromium-chromedriver

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()             # 웹 드라이버 옵션 지정
chrome_options.add_argument('--headless')              # 웹 브라우저 띄우지 않음
chrome_options.add_argument('--no-sandbox')            # 별도 웹페이지 작동 해제
chrome_options.add_argument('--disable-dev-shm-usage') # 공유 안함
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-dev-shm-usage')

: 

In [1]:
# step1.프로젝트에 필요한 패키지 불러오기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
from tqdm import tqdm


In [2]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver


In [3]:

# step2.로그인 정보 및 검색할 회사 미리 정의, 해당 회사의 리뷰 끝 페이지도 정의
usr = ""
pwd = ""
querys = ["삼성전자","LG에너지솔루션","삼성바이오로직스","SK하이닉스","LG화학","삼성전자우","삼성SDI","현대차","NAVER","기아"]
page =  1000

In [4]:

# step3.크롬드라이버 실행 및 잡플래닛 로그인

driver = set_chrome_driver()
# driver = webdriver.Chrome('chromedriver', options=chrome_options)

driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")
time.sleep(5)

login_id = driver.find_element(By.CSS_SELECTOR,"input#user_email")
login_id.send_keys(usr)

login_pwd = driver.find_element(By.CSS_SELECTOR,"input#user_password")
login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 11.8MB/s]


삼전

In [90]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/30139/reviews/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [5]:
def find_stars(object):
    a = object.get_attribute('style')[7:9]
    if a == '20':
        answer = 1
    elif a == '40':
        answer = 2
    elif a == '60':
        answer = 3
    elif a == '80':
        answer = 4
    else:
        answer = 5
    return answer

In [91]:
sample_df, sample_error = jobplanet_crawling(1,'삼성전자')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 본인들 입맛에 맞고 충성하는 사람들보다 실제 그사람의
역량을 보고 임원으로 승진시키시길
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 프리랜서로 짧게 1년식 일한거라 삼성직원들 말로는 맨날 돈 이야기 뿐임 나보다 많이 받지만 항상 다른 대기업들이랑 비교하면 많이 받는건 아니다라는둥...
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 급변하는 시장환경속에서 경쟁력을 잃지 않게 최선을 하해주셔서 감사합니다
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 어떤 타 사업부에서 이동해 온 임원이, 생활가전 사업부는 ‘패배자의 dna’ 가 있다면서 정신을 개조해야 한다고 사람들 모아놓고 얘기햇던 것이 기억난다. 아직도 경영진들이 그렇게 생각하는지 궁금함
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 보너스 제공을 경쟁사와 비교하여 총 연봉 우위로 제공되었으면함
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 합리적인 보상 체계가 있는게 직원들의 사기를 가장 잘 높일 수 있는 방법인 것 같습니다
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 보수적인 문화와 아직도 약간 남아있는 학벌문제
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 경영진에게 바라는점은 휴가를 좀 많이 줬으면 좋겠고 쉬는 시간이 있었으면 좋겠음
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 앞으로도 계속 대한민국을 대표하는 기업으로 나아가 주세요.
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 앞으로 이렇게만 쭈욱 하셨으면 좋겠네요.
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 각 직무별로 스폐셜리스트로 성장하여 회사에 기여하고, 스스로도 성취감을 느낄 수 있다면 좋겠어요.
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들의 보상에 대해 생각을 했으면 임원들만 생각하지않았으면
13번째 페이지 크롤링 중 입니다.
마지막 리뷰: 돈 좀 버는 만큼 많이 줬으면.... ps 상항 50프로폐지
14번째 페이지 크롤링 중 입니다.
마지막 리뷰

In [93]:
sample_error

[]

하닉

In [96]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [97]:
skhynix_df, skhynix_error = jobplanet_crawling(1,'SK하이닉스')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 업무 불균형이 없게 인력관리나 분배 해줄 것. 경쟁사와 같이 능력제 지향할 것. 직책자가 제 역할을 하고 있는지 역평가를 통해 상호 존중문화 만들 것.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 각 부서에 필요한 인원은 꼭 채워줬으면 좋겠음
인원이 부족
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 함께 성장했으면 좋겠다 m&a위주로 성장한 기업의 한계?
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 구성원들 입장에서 생각해줬으면 한다. 기사가 먼저나고 구성원이 뒤늦게 회사 정보를 알게 되는 것은 납득하기가 어렵다.
날짜: 2022. 12
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 실현가능항 스캐쥴링으로 로드맵제시해 주세요
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 개인적으로는 경영진분들이 내리는 결졍과 매일하는 업무가 어떻게 연관되어있는지는 이해력이 부족합니다.
날짜: 2022. 11
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 솔리다임 인수 잘한건지 모르겠음.. 다운턴 잘 극복해주길 바람
날짜: 2022. 11
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 광팔이만 하는 관리부서들이 너무 많으니까 인텔 인수같은 대실수를 하는겁니다. 물갈이 한번 해야할듯.
날짜: 2022. 10
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 소통을 하겠다고 하시는데, 경영진이 자주 바껴서 그런지 결국에는 원점으로 돌아가고있습니다.
날짜: 2022. 10
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 산업의 상황과 상관 없게 임금이 나오는 구조이면 더 좋을 것 같습니다.
날짜: 2022. 10
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 수평적 기업구조를 원함, 돈 더주시면.. 학력차별을 없애야 함
날짜: 2022. 09
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 신입 교육을 체계적으로 시켜줬으

In [99]:
skhynix_error

[]

현대자동차

In [100]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/1289/reviews/%ED%98%84%EB%8C%80%EC%9E%90%EB%8F%99%EC%B0%A8?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [101]:
hyundaicar_df, hyundai_error = jobplanet_crawling(1,"현대자동차")

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 노조 모조리 때려잡아야함.월급루팡의 근원적 존재.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 젊은 사람들을 정규직으로 많이 채용 , 계약직도 전환 하는 경우가 있었으면 좋겠음
날짜: 2023. 01
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 쪼개기계약하지말고 최소 2년은 일할수있게해줬으면 좋겠음
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 임직원에 대한 보상 체계를 개선해주시고 경영 성과에 대한 공정한 대우를 바랍니다.
날짜: 2022. 12
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 아르바이트라서 경영진에게 바라는 건 없다. 좋은 경험을 했다.
날짜: 2022. 12
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 생산직 직원들의 역량 향상을 위한 다양한 교육 프로그램이 개설 되었으면 좋겠습니다.
날짜: 2022. 12
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 조금 더 슬림한 조직, 빠른 의사결정으로 트렌드를 이끄는 회사가 되었으면
날짜: 2022. 11
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 솔직히 이제 정규직 채용좀 해야할듯 직영들 너무 고여있음
날짜: 2022. 11
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 신사업 추진 굿
요즘 나오는 자동차들 디자인 굿
품질 이슈맘 잘 해결되길
날짜: 2022. 11
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 매니저급에게도 희망을 가질수있도록,,, 복지부분에 신경을 힘써주셨으면 합니다. (기숙사, 대우 등)
날짜: 2022. 11
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 계약기간을 좀 길게 잡아주세요 2~3개월이 평균이고 최소 1달도 안되는 계약은 좀 너무하지 않나요?
날짜: 2022. 11
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 꼰대문화를 없애기 위한 노력 , 근무복 추가 지급, 보다 적극적인 안전부분에 대한 대처
날짜: 2022. 11
13번째 페이지 

In [105]:
df = pd.concat([sample_df,skhynix_df,hyundaicar_df])
df.to_csv("crawled_0117.csv",encoding='utf-8',index=False)

네이버

In [106]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/42217/reviews/%EB%84%A4%EC%9D%B4%EB%B2%84?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [107]:
naver_df, naver_error = jobplanet_crawling(1,'네이버')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들과도 성과를 나누고 해외투자도 좋지만 국내와 직원들에도 관심
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 신뢰를 바탕으로 직원들이 행복하게 일 할 수 있도록 해 주세요.
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 그냥 간단하게는 더 소통이 잘 되었으면 좋겠다
날짜: 2022. 11
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 딱히 바라는 점은 없고, 앞으로도 직원들 복지 잘 챙겨주셨으면 좋겠음
날짜: 2022. 10
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 구성원들의 이야기에 귀기울이는 점 감사합니다만
이전의 업계를 선도하는 모습이 유지되면 좋겠네요
날짜: 2022. 10
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 모두가 같은 방향을 보고 성장할 수 있도록 기회를 더 제공해주면 좋을거 같아요.
날짜: 2022. 10
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 크게 바라는 점은 없으며 직원들을 위한 복지, 보상에
앞으로도 집중해주는 것이 네이버의 경쟁력을 키우는 것이라 생각한다.
네이버야 말로 사람이 힘이 회사이다.
날짜: 2022. 09
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 평가 항목에 신경써주셨으면 좋겠습니다. 나머지는 좋습니다
날짜: 2022. 09
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 보다 투명하고 체계적인 평가 체제를 마련하고 공개하였으면함
임직원들의 사기를 증진 시킬 수 있는 방안을 모색해줬으면
날짜: 2022. 08
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 앞으로 지금처럼 회사 미래를 위해 뛰어난 인재를 고용해주길
날짜: 2022. 08
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 재도약을 위한 점검이 필요한 시점인 것 같습니다 잘 부탁 드립니다
날짜: 2022. 07
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 회사 복지차원을 지속적으로 높여주셔서 감사합니다. 앞으로도 초심을 계

In [108]:
naver_df.to_csv("naver_df.csv",encoding='utf-8',index=False)

기아

In [8]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/43248/reviews/%EA%B8%B0%EC%95%84?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
        
        if len(error_page)>30:
            time_over = False
    
    return df, error_page



In [12]:
kia_df, kia_error = jobplanet_crawling(1,'기아')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 최근 유연근무제 도입으로 근무환경 개선에 노력해 주셔서 감사드립니다.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 공정한 평가와 보상이 필요할 것 같습니다. 성과급에 대한 기준과 지급 시점에 대한 합의를 이루어 본인의 연봉이 예측가능하였으면 좋겠습니다.
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 별거없읍니다 인턴을 정직원으로 채용좀해주세요 알겠습니까?
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 제조업 특성상 어느정도 고생은 이해하는데 융통성있게는 업무를 하게 해주세요
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 높은 매출과 영업이익임에도 불구하고 낮은 성과급과 이중임금 해결좀 해주세요
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들의 말을 조금더 귀기울여 주었으면 합니다.
날짜: 2022. 11
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 비정규직으로 일하는 친구들을 정규직으로 전환시키는게 조금 더 효율적인 방법이 아닐까합니다.
날짜: 2022. 10
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 바라는 점은 없다 근로자들 입장을 많이 생각해주는 회사가 아닌가 싶음
날짜: 2022. 09
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 현재로썬 만족하고 다니고 복지도 만족하지만 더 좋은 복지를 위해 고민해줬으면 함.
날짜: 2022. 09
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 앞으로도 좋은 경영 계속 해 주시길 바랍니다.
날짜: 2022. 09
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 성과급 많이 줬으면 좋겠다. 월급을 올려주면 좋겠다.
날짜: 2022. 08
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 화성근무인원들을 위한 복지나 제도 마련이 시급함
인재 유출이 심각하다는 문제를 알면서도 방치하는 것 같아 안타까움
날짜: 2022. 08
1

카카오

In [13]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/93880/reviews/%EC%B9%B4%EC%B9%B4%EC%98%A4?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [14]:
kakao_df, kakao_error = jobplanet_crawling(1,'카카오')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 전통 대기업 루트가 아닌, 내실 있는 혁신기업으로의 길을 택했으면
날짜: 2022. 12
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 시니어들에 대한 성과를 분명히 해줄 것
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 제발… 카카오를 아낀다면 아무것도 하지 말아주세요
그냥 가만히…
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 보상 좀 더 올려줬으면 좋겠습니다 성과금이나 현금성 복지가 강화되면 좋겠어요 어차피 안 되겠지만요
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 이대로 가다가는 국민 괘씸기업으로 등극할 날이 머지않음
확장보다 내실을 다져야 함
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 인재들로 회사를 채우기 위해서는 투명한 평가와 성과제도가 합리적으로 만들어져야한다고 생각합니다. 그외 인재풀과 문화는 좋다고 새각합니다.
날짜: 2022. 10
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 사업 방향성을 명확하게 공유해야할듯..
헤매는 구성원이 많음
날짜: 2022. 09
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 학연 지연이 아직도 남아 있네요.
글로벌로 성장 하려면 해결해야할 숙제가 아닌가 합니다.
날짜: 2022. 09
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 발언에 대해 책임지는건 멋있다 하지만 그만큼 신중하게 생각해서 어나운스 하는 것도 중요한것 같다. 말 한마디에 대혼란이 있기때문에...
날짜: 2022. 09
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 회사가 커지면 사업이 잘되는 일 만큼 관리나 경영도 중요합니다. 리소스 투자가 필요합니다.
날짜: 2022. 08
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 적극적인 사업전개를 목표로한다면 일정부분의 리스크를 감수하여 속도감 있게 사업을 전개할 필요가 있어 보인다. 그렇게 된다면 훨씬 더 좋은 회사가 될 수 있을

쿠팡

In [15]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/87444/reviews/%EC%BF%A0%ED%8C%A1?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [17]:
coupang_df, coupang_error = jobplanet_crawling(1,'쿠팡')

1번째 페이지에서 오류 발생하였습니다.
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 쿠팡 파트너스 광고 좀 없앴으면 좋겠습니다... 브랜딩 되는 것이 맞는지.....ㅜㅜ
날짜: 2023. 01
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 복지를 좀 더 잘 보일 수 있도록 했으면 좋겠음. 서로 소통이 되어야 하는데 되지도 않고 가스라이팅이 있으니 독립적인 라인을 만들어 문제점이 있는 부분을 건의할 수 있도록 조치를 취해줬으면 좋겠음
날짜: 2023. 01
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 변화가 너무 많습니다..만 들어주시지는 않을 듯 ㅜㅜ 네..
날짜: 2023. 01
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 지시를 받는 사람도 지시를 내리는 사람도 이해도가 없으니 잘 될 일이 없다.
날짜: 2023. 01
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들에게 회사에 대한 자부심을 불어넣는 것은 좋으나, 큰 기업답게 가장 핵심인 현장직들의 처우개선에 더 힘써주길. 출근하는 내내 본사 앞에서 시위하는 직원들의 모습을 보고 마음이 매우 불편했음
날짜: 2023. 01
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 이대로만 더 성장해주시길 바라고 초창기 부터 종종 갔었는데 정말 많은 발전을 했다고 느낀다. 그렇지만 자동화가 될수록 개인의 업무의 강도 또한 뭔가 더 세지는 느낌이 들었다.
날짜: 2023. 01
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 프로젝트를 할때
돌다릴 두들겨 보고 건넜으면 좋겠다
날짜: 2023. 01
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 미국상장까지 하여 전직장이지만 애정이 있음. 대기업으로 성장했으니 승승장구하길
날짜: 2023. 01
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 바란다고 바뀌는것도 아니라 안 적을 거임
날짜: 2023. 01
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 생산직 사람들에게 대우가 박한 감이 있어요. 여름에 덥고 겨울에 춥고 근무환경이 너무 고됩니

KT

In [7]:
def jobplanet_crawling(num,company):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/53191/reviews/%EC%BC%80%EC%9D%B4%ED%8B%B0?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
        
        if len(error_page)>30:
            time_over=False
        
    
    return df, error_page



In [8]:
kt_df, kt_error =jobplanet_crawling(1,'KT')

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 신규 사업을 적극적으로 추진하나 실패하는 경우가 많음. 기존 사업에 집중하면 더 좋은 실적이 예상이됨.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 고생한만큼 임직원들에게 성과급, 일시금 등 인센티브를 많이주세요. 연봉인상률이 낮아 퇴사자가 많은거같네요. 직원수가 너무 많습니다. 조직 인원을 절반으로 슬림화 할 필요가있습니다
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 매번 말로만 DIGICO KT외치지 마시고 뭔가 확실한 대응과 그에 대한 정확한 보상을 해주세요.
날짜: 2022. 11
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 회사의 미래를 위해서라도 직원들의 가치가 높아질 수 있도록 신경을 써주셨으면 좋겠습니다
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 대리과장급 퇴사했다고 신입사원 머리수 맞춰 뽑는건 의미없습니다.
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 회사의 발전을 위해 매진해 주시길 부탁 드립니다
날짜: 2022. 11
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 임원직에 연봉 인상도 중요하지만
밑에서부터 고생하는 최일선 직원들에 노고도 알아주어야함
날짜: 2022. 10
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 주니어들이 왜 그렇게 많이 떠나는지 자각하셨으면 좋겠습니다.
오너쉽 없는게 너무 보여요 경영진들
날짜: 2022. 09
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 젊은 사람들을 위한 문화를 많이 도입해주세요. 인재개발에 힘써주세요
날짜: 2022. 09
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 아무것도 안하면서 말로만 지시하는 고위직 고인물들에게 일 좀 시켰으면 해요....
날짜: 2022. 09
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 당연한 얘기지만 연봉이랑 복지에 좀 더 힘써줬으면 함
날짜: 2022. 08
12번째 페이지 크롤링 중 입

In [17]:
kt_df.to_csv("kt_df.csv",encoding='utf-8',index=False)

## 함수 최종 정리


In [6]:
def jobplanet_crawling(num,company,url):
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while time_over:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get(url.format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(company)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if int(user_info[4*i+3].text.replace(". ","")) <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            print("날짜: {}".format(user_info[4*i+3].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
        
        if len(error_page)>30:
            time_over=False
        
    
    return df, error_page



In [11]:
def sample_crawl(num,company,url):
    driver.get(url.format(num))
    time.sleep(3)

In [20]:
crawl_list = [(1,'CJ제일제당',"https://www.jobplanet.co.kr/companies/23189/reviews/cj%EC%A0%9C%EC%9D%BC%EC%A0%9C%EB%8B%B9?page={}"),
              (1,'넥슨코리아','https://www.jobplanet.co.kr/companies/83498/reviews/%EB%84%A5%EC%8A%A8%EC%BD%94%EB%A6%AC%EC%95%84?page={}'),
              (1,'대우조선해양','https://www.jobplanet.co.kr/companies/39528/reviews/%EB%8C%80%EC%9A%B0%EC%A1%B0%EC%84%A0%ED%95%B4%EC%96%91?page={}'),
              (1,'KCC',"https://www.jobplanet.co.kr/companies/53607/reviews/%EC%BC%80%EC%9D%B4%EC%94%A8%EC%94%A8?page={}")]

In [21]:
for i in crawl_list:
    df, error_df = jobplanet_crawling(i[0],i[1],i[2])
    df.to_csv('{}_df.csv'.format(i[1]),index=False,encoding='utf-8')
    time.sleep(5)


1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 경영진에게 바라는 점으로는 좀더 넓게보고 구시대적인 마인드좀 버렸으면 합니다.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 합리적인 보상체계나 근무환경에 대해서 더힘써주시길 바라겠습니다.
날짜: 2023. 01
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 인턴하고도 떨어지는 사람이 많은데 조금이라도 전환율 올릴수는 없는지
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 돈을 좀만 더줬으면 좋겠어요 지역은 어쩔수 없지만
날짜: 2022. 12
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 아르바이트로서 경험 쌓기에는 좋은 곳이였어서 딱히 바라는 점은 없습니다.
날짜: 2022. 12
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 합리적인 보상체계를 마련 해 주었으면 합니다.
날짜: 2022. 12
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 고인물들은 좀 내치세요 제발 이런애들 갈곳도없어서 썩고있는데 걍 내치세요 제발
날짜: 2022. 12
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들 처우를 생각해주시면 이직하는 사원들이 적어질거에여
날짜: 2022. 11
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 근무체계(급여, 복지 등)은 변함없이 그만두면 새로 뽑으면 된다는 느낌인데 이제는 이런 이미지가 쇄신되는 운영을 했으면 좋겠다.
날짜: 2022. 11
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 돈 아끼지 말고 좀 쓰세요. 설비 고장나면 스페어가 없어서 설비 보수를 못한다는 게 말이 안됩니다. 그리고 구성원들한테 일 많이 주고 싶으면 그에 타당한 임금을 주세요. 포괄임금제하면서 다 빨아먹으려고 하지말고
날짜: 2022. 11
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: … 다들 아실거라고 생각하는데, 무엇이 중요하고 맞는가를 판단하시길. 그저 입맛맞추기 급급해서 내일만 사는 사람들처럼 오너가 손가락끝을 보지말고, 손가락이 가르치는 

## 코드 

In [7]:
crawl_list = [(1,'포스코','https://www.jobplanet.co.kr/companies/50130/reviews/%ED%8F%AC%EC%8A%A4%EC%BD%94?page={}'),
              (1,'KB국민은행','https://www.jobplanet.co.kr/companies/85166/reviews/%EA%B5%AD%EB%AF%BC%EC%9D%80%ED%96%89?page={}'),
              (1,'하나은행','https://www.jobplanet.co.kr/companies/49203/reviews/%ED%95%98%EB%82%98%EC%9D%80%ED%96%89?page={}'),
              (1,'한국전력공사','https://www.jobplanet.co.kr/companies/3150/reviews/%ED%95%9C%EA%B5%AD%EC%A0%84%EB%A0%A5%EA%B3%B5%EC%82%AC?page={}'),
              (1,'엔씨소프트','https://www.jobplanet.co.kr/companies/64418/reviews/%EC%97%94%EC%94%A8%EC%86%8C%ED%94%84%ED%8A%B8?page={}'),
              (1,'두산에너빌리티','https://www.jobplanet.co.kr/companies/36828/reviews/%EB%91%90%EC%82%B0%EC%97%90%EB%84%88%EB%B9%8C%EB%A6%AC%ED%8B%B0?page={}'),
              (1,'대한항공','https://www.jobplanet.co.kr/companies/81661/reviews/%EB%8C%80%ED%95%9C%ED%95%AD%EA%B3%B5?page={}'),
              (1,'아모레퍼시픽','https://www.jobplanet.co.kr/companies/68464/reviews/%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD?page={}')]

In [8]:
for i in crawl_list:
    df, error_df = jobplanet_crawling(i[0],i[1],i[2])
    df.to_csv('{}_df.csv'.format(i[1]),index=False,encoding='utf-8')
    time.sleep(5)

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 현장과 소통이 필요합니다.

중간 관리자들이 현장의 소리를 왜곡을 시킵니다.
날짜: 2022. 12
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원 애사심 고취를 위해 연봉 복리후생에 힘써주세요.
태풍피해에도 몇달만에 파업도 없이 묵묵하게 가동시키는 직원들을 위해서요!!!!!
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원 처우개선에 힘을 써야 할 시기에 경영진의 성과급 잔치만 하는중
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 안전시설투자 복지대폭강화 지방근무하는대신 더욱더높은연봉
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 다른 기업 어떻게 하는지 보고 직원들 사기 진작할수있는 방안도 좀 생각을
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 대기업 답게 계약직에게도 타당한 대우가 필요로 함, 사원들에게도 고위직 평가제도가 필요하다고 봄
날짜: 2022. 10
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 업무의 간소화가 필요한 거 같습니다. 이직자들이 너무나 많은데 문제가 있는거 같아요
날짜: 2022. 10
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 실적이 좋으면 직원에게도 적극적인 보상이 주어져야하며 기존에 있는 복지를 없앨것이 아니라 더 추가해줘야함
또한 경영진이 솔선수범하여 위기극복을 위한 의지를 보여주어야겠으며, 불필요한 투자를 할 것이아니라 설비 개선 등 투자에 적극 동참할 것을 요구함
날짜: 2022. 09
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 본인들 임기동안만 한통 크게 해먹고 빠질 생각하지마시고 진정으로 회사를 생각했으면 좋겠음
날짜: 2022. 09
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 직원들만 불만을 듣지만 말고 합리적인 해결책과 방법을 제시했으면 함. 본인들의 자리와 미래만 신경쓰지 말고 회사와 직원들의 미래도 신경쓰기 바람.
날짜: 2022. 0

In [9]:
crawl_list = [(1,'삼성바이오로직스',"https://www.jobplanet.co.kr/companies/90162/reviews/%EC%82%BC%EC%84%B1%EB%B0%94%EC%9D%B4%EC%98%A4%EB%A1%9C%EC%A7%81%EC%8A%A4?page={}"),
              (1,'LG화학','https://www.jobplanet.co.kr/companies/63852/reviews/lg%ED%99%94%ED%95%99?page={}')]

In [10]:
for i in crawl_list:
    df, error_df = jobplanet_crawling(i[0],i[1],i[2])
    df.to_csv('{}_df.csv'.format(i[1]),index=False,encoding='utf-8')
    time.sleep(5)

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 사내대출이 필요해보임. 거주지 문제가 해결 안되면 오래 일할 슌 없음..
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 있는 사람 갈아넣지 말고 인원은 좀 많이 뽑거나 효율적으로 운행해야 될듯
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 기업 문화 하나는 진짜 좋습니다. 직원들이 꾸준히 발전할 수 있게 자기계발 할 수 있는 시스템도 잘되어있구요. 바라는 점은 잘 모르겠어요.
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 바래봤자 피드백이 없으니 바라지도 않습니다.
이러니 일하는 사람들은 다 퇴사하지..
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 경영진에게 바라는 점은 딱히 없습니다
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 현업의 이야기를 더 들어주면 좋겠고 복지도 좋지만 연봉도 신경을 써주면 좋겠다.
날짜: 2022. 11
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 그룹사 평가기간에만 임직원들 고충 듣는 척 하지 말고 평소에도 좀 들어줬으면
날짜: 2022. 10
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 교대수당 더챙겨주셔야합니다 교대근무자들 정말 힘들게일합니다
날짜: 2022. 10
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 높은 연봉인상률,워라밸개선,젊은사람에 맞춘 복지 개선도 필요
날짜: 2022. 10
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 신입보다는 경력뽑아주길 특히 qa
너무빡빡한 스케줄에 신입을 왜갈아넣는건지..
날짜: 2022. 09
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 성과금 및 직원 처우개선이 시급하다. 직원교육도 필요함
날짜: 2022. 09
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 계약직에게 계약한 업무시간, 할당된 업무만 시켰으면 좋겠다
자기들 일손 급할땐 아쉬운소리 하면서 잔업요구하고, 당장 할수있는 업무

In [11]:
crawl_list = [(1,'현대모비스','https://www.jobplanet.co.kr/companies/1415/reviews/%ED%98%84%EB%8C%80%EB%AA%A8%EB%B9%84%EC%8A%A4?page={}'),
              (1,'LG전자','https://www.jobplanet.co.kr/companies/19514/reviews/lg%EC%A0%84%EC%9E%90?page={}'),
              (1,'삼성생명보험','https://www.jobplanet.co.kr/companies/30195/reviews/%EC%82%BC%EC%84%B1%EC%83%9D%EB%AA%85%EB%B3%B4%ED%97%98?page={}'),
              (1,'LG생활건강','https://www.jobplanet.co.kr/companies/63861/reviews/%EC%97%98%EC%A7%80%EC%83%9D%ED%99%9C%EA%B1%B4%EA%B0%95?page={}')
              ]

In [12]:
for i in crawl_list:
    df, error_df = jobplanet_crawling(i[0],i[1],i[2])
    df.to_csv('{}_df.csv'.format(i[1]),index=False,encoding='utf-8')
    time.sleep(5)

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 다른 것보다, 시스템 개선이 정말 필요함. 업무 비효율의 거의 대부분이 시스템 문제라고 생각
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 깨어있는 마인드를 가지셨으면,,,
요즘 트렌드를 따라서 변화를 추구해주세요
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 미래 비젼을 직원에게 제시하고 직원들의 역량개발에 힘썼으면 좋겠습니다.
날짜: 2022. 12
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 아무리 바쁘고 뭐 해도 최대 수량 +400 +600은 좀 에바임
날짜: 2022. 11
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 업무 담당자를 명확하게 알 수 있는 프로세스를 갖추어 줬으면 좋겠음,
날짜: 2022. 11
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 메뉴얼이 있었으면 좋겠습니다. 그리고 본사에서 모른다고 계약직에게 되려 물어보는건 무슨 상황인지 정말 이해가 안갔습니다.
날짜: 2022. 11
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 계약직도 업무평그로 전환 가능한 제도거 생기면 좋을꺼 같음
날짜: 2022. 10
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 현대차 기조실로부터의 말도안되는 지시사항좀 그만...
날짜: 2022. 10
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 불필요한 문서작성 최소화
휴게실 필요.
날짜: 2022. 10
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 우수직원 및 일부 부서에 대한 인센티브 지급제도가 없으면, 공산주의나 다를바 없음
날짜: 2022. 10
11번째 페이지 크롤링 중 입니다.
마지막 리뷰: 경영진에 따라 사업 계획이나 방향성, 내부 분위기가 좌우되는 것을 느꼈습니다. 이 부분을 최소화 한다면 보다 좋은 기업문화가 형성되지 않을까 싶습니다.
날짜: 2022. 09
12번째 페이지 크롤링 중 입니다.
마지막 리뷰: 제에에발 연구원들좀 생각해주세요. 자회사 시키지말고 그냥 지금 있는

In [9]:
crawl_list = [(1,'라인','https://www.jobplanet.co.kr/companies/89255/reviews/%EB%9D%BC%EC%9D%B8%ED%94%8C%EB%9F%AC%EC%8A%A4?page={}'),
              (1,'롯데쇼핑','https://www.jobplanet.co.kr/companies/89356/reviews/%EB%A1%AF%EB%8D%B0%EC%87%BC%ED%95%91-%EB%B0%B1%ED%99%94%EC%A0%90%EC%82%AC%EC%97%85%EB%B3%B8%EB%B6%80?page={}')]

In [10]:
for i in crawl_list:
    df, error_df = jobplanet_crawling(i[0],i[1],i[2])
    df.to_csv('{}_df.csv'.format(i[1]),index=False,encoding='utf-8')
    time.sleep(5)

1번째 페이지 크롤링 중 입니다.
마지막 리뷰: 크게 바라는 점은 없습니다. 지금처럼 많은 관심 가져주세요! 많이 배웠습니다.
날짜: 2023. 01
2번째 페이지 크롤링 중 입니다.
마지막 리뷰: 업무 속도는 빠르지만 의사결정 속도가 느리기때문에, 시장에 진출할 때 핵심적인 타이밍을 다 놓치는 것 같음. 차라리 의사결정 속도를 빠르게 해줬으면 함
날짜: 2022. 12
3번째 페이지 크롤링 중 입니다.
마지막 리뷰: 주기적인 all hands를 통해서 경영방침이 주기적으로 공유되어서 좋습니다만, 조금만 더 많이 노력해주세요
날짜: 2022. 11
4번째 페이지 크롤링 중 입니다.
마지막 리뷰: 거버넌스 정비를 통해 업무의 속도감을 올렸으면
더 활발한 소통 문화
날짜: 2022. 10
5번째 페이지 크롤링 중 입니다.
마지막 리뷰: 아르바이트로 1년 미만 근무했기 때문에 별도로 경영진에게 바라는 점 없음.
날짜: 2022. 09
6번째 페이지 크롤링 중 입니다.
마지막 리뷰: 사업을 추진하는데 있어 선택과 집중을 잘 고려하면 좋겠습니다.
날짜: 2022. 08
7번째 페이지 크롤링 중 입니다.
마지막 리뷰: 야근이 중요한게 아닙니다 사람들 목소리를 들어주세요….
날짜: 2022. 07
8번째 페이지 크롤링 중 입니다.
마지막 리뷰: 경영진의 방향은 뭔가요?직원들과 소통해주세요
묵묵히 일하면 무시당하고 메신저 다이렉트로 보내면 허심탄회한 직원이라는 기준은 참 어렵습니다
임원이 일본에 있으니 한없이 현황에 눈감고 계시는데 직시해주세요
날짜: 2022. 06
9번째 페이지 크롤링 중 입니다.
마지막 리뷰: 지금보다 더 시람들을 잘 챙겼으면 좋겠다 IT기업들의 인재 모시기 경쟁으로 좋은 동료들이 빠져나가는 것에 경각심을 가졌으면 좋겠다
날짜: 2022. 05
10번째 페이지 크롤링 중 입니다.
마지막 리뷰: 애정을 갖고 있는 직원들이 많으니 앞으로도 계속해서 성장할 수 있도록 사업에 대한 고민을 계속 해주면 좋겠습니다!
날짜: 2022. 04
11번째 페이지 크롤링 

## 구분선

In [66]:
    df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})
    time_over = True
    page_num = num
    error_page = []

    # list_company = []
    # list_div = []
    # list_cur = []
    # list_date =[]
    # list_stars = []
    # list_summary = []
    # list_merit = []
    # list_disadvantages = []
    # list_managers =[]

    while page_num <5:
        
        temp_company = []
        temp_div = []
        temp_cur = []
        temp_date = []
        temp_stars = []
        temp_summary = []
        temp_merit = []
        temp_disadvantages = []
        temp_managers = []
        
        try:
            driver.get("https://www.jobplanet.co.kr/companies/373976/reviews/%EC%97%98%EC%A7%80%EC%97%90%EB%84%88%EC%A7%80%EC%86%94%EB%A3%A8%EC%85%98?page={}".format(page_num))
            time.sleep(3)

            
            user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")
            stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
            summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")
            review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

            for i in range(5):
                temp_company.append(query)
                temp_div.append(user_info[4*i].text)
                temp_cur.append(user_info[4*i+1].text)
                temp_date.append(user_info[4*i+3].text)
                temp_stars.append(find_stars(stars[i]))
                temp_summary.append(summary[i].text)
                temp_merit.append(review[3*i].text)
                temp_disadvantages.append(review[3*i+1].text)
                temp_managers.append(review[3*i+2].text)
                
                
            if user_info[4*i+3].text.replace(". ","") <= 201612:
                time_over = False 
                
            print("{}번째 페이지 크롤링 중 입니다.".format(page_num))
            print("마지막 리뷰: {}".format(review[3*i+2].text))
            
            temp_df = pd.DataFrame()
            temp_df['회사'] = pd.Series(temp_company)
            temp_df['날짜'] = pd.Series(temp_date)
            temp_df['직무'] = pd.Series(temp_div)
            temp_df['재직여부'] = pd.Series(temp_cur)
            temp_df['별점'] = pd.Series(temp_stars)
            temp_df['요약'] = pd.Series(temp_summary)
            temp_df['장점'] = pd.Series(temp_merit)
            temp_df['단점'] = pd.Series(temp_disadvantages)
            temp_df['경영진에게 바라는 점'] = pd.Series(temp_managers)
            df = pd.concat([df, temp_df])
            
            
        except:
            print("{}번째 페이지에서 오류 발생하였습니다.".format(page_num))
            error_page.append(page_num)
        
        page_num += 1
    
    return df, error_page



In [18]:
jobplanet_df = pd.concat([jobplanet_df, total_data])
jobplanet_df.head()

,장점,경영진에게 바라는 점,직무,별점,날짜,요약,단점,회사,재직여부
0,PS와 삼성맨이라는 약간의 자부심. 대기업의 복지 정도..,"반도체 한해서는 그냥 수율 지상주의, 경영진의 의견이 반영이 된건지는 모르겠지만, ...",생산관리/품질관리,2점,2023. 01,"""대기업이라 부서 별 극과 극. 좋은 데가면 편하고 구린 데가면 개고생한다.""",반도체 F직군은 그냥 헬이다. 대체 초 가능 인력으로 업무 방식도 주먹구구. 이슈 ...,삼성전자,전직원
1,"연차를 눈치 안보고 쓸 수 있고 급여수준이 괜찮고, 조직문화 나쁘지않음",좀더 발전적인.방향을 고민했으면 좋겠다.,기획/경영,5점,2023. 01,"""커리어를 쌓기 좋고 급여수중 높다. 대체적으로 만족스러움""","딱히 찾을순 없지만, 워라밸이 깨지는 부바부 구조인 것 같다. 이 외에는 딱히 없음",삼성전자,전직원
2,"일단 주변인들에게 삼성에 근무한다는것으로도 신원이 확실히 보장되는 느낌, 누릴수있는...",가끔은 조금 현상황에 대해서 안주하는 느낌이 들었습니다. 위기의식이 필요하고 좀더 ...,개발,4점,2023. 01,"""대한민국 최고의 기업에서 일한다는것 자체가 좋았습니다. 전반적인 직원들 능력이 매...","조금은 경직된 조직문화, 효울을 강조하지만 생각보다는 효율적이지 못한 시스템이 아쉬...",삼성전자,현직원
3,연봉과 자잘한 복지가 좋고 다른사람에게 굳이 설명 필요 없다,연봉 인상 및 의사소통 강화가필요합니다 열린귀로 소통이 필요합니다,생산관리/품질관리,4점,2023. 01,"""개발자가 다양하게 의견을 존중받는곳이므로 추천할만함""",경영진과의 의사소통 부재로 답답할때가 있다 그래서 이것을 단점으로 꼽습니다,삼성전자,현직원
4,하루 아침 점심 저녁 무료 제공\n경기 남부 및 서울 전역 셔틀 버스\n수평적인 분...,경영 정보 등을 직원들에게도 투명하게 공개하였으면 좋겠습니다\n직원들에게 더 많은 ...,개발,5점,2023. 01,"""연봉과 복지가 매우 좋은 회사\n국내 탑 대기업이자 글로벌 회사에 다닌다는 자부심...",과제 프로젝트에 따른 업무 load가 있음\n받는만큼 일해야하는 책임감,삼성전자,현직원


In [31]:
jobplanet_df.to_csv("jobplanet_df.csv",encoding='utf-8',index=False)

In [ ]:
# jobplanet_df = pd.DataFrame(columns={'회사','날짜','직무','재직여부','별점','요약','장점','단점','경영진에게 바라는 점'})

for query in querys:
  time_over = False
        # step4.원하는 회사의 리뷰 페이지까지 이동
  search_query = driver.find_element(By.CSS_SELECTOR,"input#search_bar_search_query")
  search_query.send_keys(query)
  search_query.send_keys(Keys.RETURN)
  time.sleep(3)

  driver.find_element(By.CSS_SELECTOR,"a.tit").click()
  time.sleep(15)
  try: 
    driver.find_element(By.CSS_SELECTOR,"button.btnㅠ_close_x_ty1 ").send_keys(Keys.ENTER)
    time.sleep(3)
  except:
    pass



  # step6.크롤링한 정보를 담을 리스트명 정의
  list_company = []
  list_div = []
  list_cur = []
  list_date =[]
  list_stars = []
  list_summary = []
  list_merit = []
  list_disadvantages = []
  list_managers =[]
  for i in tqdm(range(140)):
    try:
        driver.find_element(By.CSS_SELECTOR,"a.btn_pgnext").click()
        time.sleep(1)
    except:
        pass


  # step7.원하는 회사의 직무/근속여부/일시/요약/평점/장점/단점/경영진에게 바라는 점 크롤링 (for문으로 반복)
  for i in tqdm(range(0,page)): 
      if time_over==True:
          break

      #직무, 근속여부, 일시
      user_info = driver.find_elements(By.CSS_SELECTOR,"span.txt1")

      count = int(len(user_info)/4)
      # print(count)
      
      for j in range(count):
          list_company.append(query)

      list_user_info = []

      for j in user_info:
          list_user_info.append(j.text)

      for j in range(count):            #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
          a = list_user_info[4*j]
          list_div.append(a)
          
          b = list_user_info[4*j+1]
          list_cur.append(b)

          c = list_user_info[4*j+3]
          if int(c.replace(". ","")) <= 201612:
              time_over = True
          list_date.append(c)

      #별점
      stars = driver.find_elements(By.CSS_SELECTOR,"div.star_score")
      for j in stars:
          a = j.get_attribute('style')
          if a[7:9] == '20':
              list_stars.append("1점")
          elif a[7:9] == '40':
              list_stars.append("2점")
          elif a[7:9] == '60':
              list_stars.append("3점")
          elif a[7:9] == '80':
              list_stars.append("4점")
          else:
              list_stars.append("5점")
          
      #요약 정보
      summary = driver.find_elements(By.CSS_SELECTOR,"h2.us_label")

      for j in summary:
          list_summary.append(j.text)
      
      #장점, 단점, 경영진에게 바라는 점
      list_review = []

      review = driver.find_elements(By.CSS_SELECTOR,"dd.df1")

      for j in review:
          list_review.append(j.text)

      for j in range(count):            #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
          a = list_review[3*j]
          list_merit.append(a)
          
          b = list_review[3*j+1]
          list_disadvantages.append(b)

          c = list_review[3*j+2]
          list_managers.append(c)

      # 다음 페이지 클릭 후 for문 진행, 끝 페이지에서 다음 페이지 클릭 안되는 것 대비해서 예외처리 구문 추가
      try:
          driver.find_element(By.CSS_SELECTOR,"a.btn_pgnext").click()
          time.sleep(5)
      except:
          break


  # step8.pandas 라이브러리로 표 만들기
  total_data = pd.DataFrame()
  total_data['회사'] = pd.Series(list_company)
  total_data['날짜'] = pd.Series(list_date)
  total_data['직무'] = pd.Series(list_div)
  total_data['재직여부'] = pd.Series(list_cur)
  total_data['별점'] = pd.Series(list_stars)
  total_data['요약'] = pd.Series(list_summary)
  total_data['장점'] = pd.Series(list_merit)
  total_data['단점'] = pd.Series(list_disadvantages)
  total_data['경영진에게 바라는 점'] = pd.Series(list_managers)
  jobplanet_df = pd.concat([jobplanet_df, total_data])




# step9.엑셀 형태로 저장하기

# step10.크롬 드라이버 종료
driver.quit()

In [ ]:
total_data.to_csv("잡플래닛 리뷰_" + query + ".csv" ,index=True, encoding='utf-8-sig')


False